In [1]:

%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import place_cell_base as pl
import os
import helper_functions as hf
import scipy.stats as stats

path = os.path.abspath(os.path.dirname(pl.__file__))



In [15]:

mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

# modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)

modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)

# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 40,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 20, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 60, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 25,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 15, s = 5)


modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))

modulated_timestamps = np.sort(np.random.choice(modulated_timestamps,50,replace=False))
# modulated_timestamps = np.sort(np.random.choice(I_timestamps,50,replace=False))

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))


place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent = 0.1,
                          minvisits = 1, speed_threshold = 5, nbins_cal = 10, num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, 
                          nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50)

PlaceCellObject = place_cell.run_placeMetrics(calcium_imag,timevector,x_coordinates,y_coordinates)


plt.figure()
cnt = plt.contourf(PlaceCellObject['x_center_bins'],PlaceCellObject['y_center_bins'],PlaceCellObject['place_field_smoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title(f"MI = {PlaceCellObject['mutualInfo_zscored']:0.3f} ; Events = {calcium_events}")
plt.show()


/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:178: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:

array_output = grid_spatial_autocorrelation(PlaceCellObject['place_field_smoothed'])
array_output


plt.figure()
cnt = plt.contourf(array_output,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()


<ipython-input-38-61bb9e2df660>:53: RuntimeWarning: invalid value encountered in double_scalars
  out_mat[i,j] = (n*sigma_x_y - sigma_x*sigma_y) / (np.sqrt(n*sigma_x2-sigma_x**2) * np.sqrt(n*sigma_y2-sigma_y**2));


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
autoCorrRot = rotate(autoCorr, angle=30, reshape=False)


plt.figure()
cnt = plt.contourf(autoCorrRot,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
from scipy.ndimage.interpolation import rotate


array_output_zeroed = np.copy(array_output)
array_output_zeroed[np.isnan(array_output_zeroed)] = 0
autoCorr = np.copy(array_output_zeroed)
da = 3
angles = list(range(0, 180+da, da))
crossCorr = []
# Rotate and compute correlation coefficient
for angle in angles:
    autoCorrRot = rotate(autoCorr, angle, reshape=False)
    C = np.corrcoef(np.reshape(autoCorr, (1, autoCorr.size)),
        np.reshape(autoCorrRot, (1, autoCorrRot.size)))
    crossCorr.append(C[0, 1])

max_angles_i = (np.array([30, 90, 150]) / da).astype(int)
min_angles_i = (np.array([60, 120]) / da).astype(int)

maxima = np.max(np.array(crossCorr)[max_angles_i])
minima = np.min(np.array(crossCorr)[min_angles_i])
G = minima - maxima
G

        

-0.8396819379204197

In [29]:

plt.figure()
plt.plot(crossCorr)
plt.show()


<ipython-input-29-233a78f9d8ba>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
import scipy. signal as sig


array_output = sig.correlate2d(PlaceCellObject['place_field_smoothed'],PlaceCellObject['place_field_smoothed'])
array_output


plt.figure()
cnt = plt.contourf(array_output,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
def grid_spatial_autocorrelation(calcium_mean_occupancy_smoothed):

    mat1 = calcium_mean_occupancy_smoothed
    mat2 = calcium_mean_occupancy_smoothed

    [ma,na] = np.shape(mat1)
    [mb,nb] = np.shape(mat2)

    mc = np.max([ma+mb-1,ma,mb])
    nc = np.max([na+nb-1,na,nb])
    out_mat = np.nan*np.zeros([mc,nc])


    i_size = mat2.shape[0]
    j_size = mat2.shape[1]

    mat = np.copy(mat1);
    h = np.copy(mat2);
    l = 1

    npad_ij = np.ceil(np.array(np.shape(h))/l).astype(int);
    npad_i = npad_ij[0];
    npad_j = npad_ij[1];
    in_size = np.array(np.shape(mat));

    out_size = in_size + [2*npad_i, 2*npad_j];
    work_mat = np.nan * np.zeros(out_size);
    work_mat[(npad_i):(npad_i+in_size[0]),(npad_j):(npad_j+in_size[1])] = mat

    for i in range(np.shape(out_mat)[0]):
        for j in range(np.shape(out_mat)[1]):


            win1 = (np.arange(npad_i+i-np.floor(i_size),(npad_i+i))).astype(int)
            win2 = (np.arange(npad_j+j-np.floor(j_size),(npad_j+j))).astype(int)

            sub_mat = work_mat[win1,:][:,win2]; 
            nan_sub_mat=sub_mat*mat2;                                             
            notnan_inds = ~np.isnan(nan_sub_mat)

            n= np.sum(notnan_inds);

            if n < 20:
                out_mat[i,j] = np.nan;

            sigma_x_y = np.sum(nan_sub_mat[notnan_inds]);
            sigma_x =   np.sum(sub_mat[notnan_inds]);
            sigma_y =   np.sum(mat2[notnan_inds]);
            sigma_x2 =  np.sum(sub_mat[notnan_inds]**2);
            sigma_y2 =  np.sum(mat2[notnan_inds]**2);

            out_mat[i,j] = (n*sigma_x_y - sigma_x*sigma_y) / (np.sqrt(n*sigma_x2-sigma_x**2) * np.sqrt(n*sigma_y2-sigma_y**2));

    return out_mat




In [58]:

mat1 = PlaceCellObject['place_field_smoothed']
mat2 = PlaceCellObject['place_field_smoothed']


In [63]:

[ma,na] = np.shape(mat1)
[mb,nb] = np.shape(mat2)
nb

50

In [67]:

mc = np.max([ma+mb-1,ma,mb])
nc = np.max([na+nb-1,na,nb])
out_mat = np.nan*np.zeros([mc,nc])

out_mat.shape


(99, 99)

In [69]:

i_size = mat2.shape[0]
j_size = mat2.shape[1]
j_size

50

In [73]:


mat = np.copy(mat1);
h = np.copy(mat2);
l = 1



In [74]:
npad_ij = np.ceil(np.array(np.shape(h))/l).astype(int);
npad_ij

array([50, 50])

In [81]:

npad_i = npad_ij[0];
npad_j = npad_ij[1];

in_size = np.array(np.shape(mat));
in_size

array([50, 50])

In [82]:
out_size = in_size + [2*npad_i, 2*npad_j];
out_size

array([150, 150])

In [89]:
work_mat = np.nan * np.zeros(out_size);
work_mat.shape
work_mat[(npad_i):(npad_i+in_size[0]),(npad_j):(npad_j+in_size[1])] = mat


In [90]:

plt.figure()
plt.pcolormesh(work_mat)
plt.colorbar()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:

for i in range(np.shape(out_mat)[0]):
    for j in range(np.shape(out_mat)[1]):


        win1 = (np.arange(npad_i+i-np.floor(i_size),(npad_i+i))).astype(int)
        win2 = (np.arange(npad_j+j-np.floor(j_size),(npad_j+j))).astype(int)

        sub_mat = work_mat[win1,:][:,win2]; 
        nan_sub_mat=sub_mat*mat2;                                             
        notnan_inds = ~np.isnan(nan_sub_mat)

        n= np.sum(notnan_inds);

        if n < 20:
            out_mat[i,j] = np.nan;

        sigma_x_y = np.sum(nan_sub_mat[notnan_inds]);
        sigma_x =   np.sum(sub_mat[notnan_inds]);
        sigma_y =   np.sum(mat2[notnan_inds]);
        sigma_x2 =  np.sum(sub_mat[notnan_inds]**2);
        sigma_y2 =  np.sum(mat2[notnan_inds]**2);

        out_mat[i,j] = (n*sigma_x_y - sigma_x*sigma_y) / (np.sqrt(n*sigma_x2-sigma_x**2) * np.sqrt(n*sigma_y2-sigma_y**2));






<ipython-input-52-efbca616356b>:50: RuntimeWarning: invalid value encountered in double_scalars
  out_mat[i,j] = (n*sigma_x_y - sigma_x*sigma_y) / (np.sqrt(n*sigma_x2-sigma_x**2) * np.sqrt(n*sigma_y2-sigma_y**2));


In [54]:


plt.figure()
plt.pcolormesh(out_mat)
plt.colorbar()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …